<a href="https://colab.research.google.com/github/kroncatti/complex-data-mining/blob/master/inf-0617-big-data/assign1-sentiment-analysis-spark/Big_Data_013_Trabalho_1_Sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Integrantes do grupo:

- Kaleb Roncatti de Souza
- Nelson Gomes Brasil Junior

**Instruções iniciais**

*   Abra os links dos dados: 
    * https://tinyurl.com/bd-friends
*   Clique em "Adicionar atalho ao Drive"


# Solução

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install nltk twython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!python -m nltk.downloader vader_lexicon

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
!wget -q https://downloads.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install findspark pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 48 kB/s 
     |████████████████████████████████| 199 kB 48.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=7893a6d4142fdf5a4593da0a1bcee2ed6ac9a9cf83177a181ac4b354192b5381
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [5]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/default-java
%env SPARK_HOME=/content/spark-3.2.2-bin-hadoop3.2

env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/default-java
env: SPARK_HOME=/content/spark-3.2.2-bin-hadoop3.2


In [6]:
import findspark
findspark.init("/content/spark-3.2.2-bin-hadoop3.2")

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

from datetime import datetime

appName = 'Big Data'
master = 'local[*]'

spark = SparkSession.builder     \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [72]:
input_data = spark.sparkContext.textFile('file:///content/drive/My Drive/friends')

In [73]:
from nltk.sentiment import SentimentIntensityAnalyzer

sentiment = SentimentIntensityAnalyzer()
characters = ['monica', 'chandler', 'ross', 'rachel', 'joey', 'phoebe', 
              'gunther', 'janice', 'richard', 'emily']


In [74]:
print(input_data.take(10))
log = f"Total of records : {input_data.count()}"
print(log)
log = f"Number of partitions: {input_data.getNumPartitions()}"
print(log)

['1001 - The One After Joey And Rachel Kiss', 'Written by: Andrew Reich & Ted Cohen', 'Transcribed by: Coffee Mug', '', "[Scene: Barbados, Monica and Chandler's Room. They both enter from Ross's room. Monica still has her big, frizzy hair.]", '', 'Monica: Oh, the way you crushed Mike at ping pong was such a turn-on.You wanna...? (plays with her finger on Chandlers chest)', '', "Chandler: You know, I'd love to, but I'm a little tired.", '']
Total of records : 140563
Number of partitions: 227


In [75]:
def line_sentiment(line):
  line = line.lower()
  try:
    character, phrase = line.split(':', 1)
    # Checking if the current character is on our list
    if character in characters:
      # Returning (Key, (Polarity, 1))
      yield(character, (float(sentiment.polarity_scores(phrase)["compound"]), 1))

  except Exception as e:
    # In a real case scenario, we would most probably try to check what we are ignoring
    pass
  

In [76]:
s = input_data.flatMap(line_sentiment)

In [78]:
# Checking the data
print(s.take(10))
sample = s.take(1) 
print(f"Key: {sample[0][0]}")
print(f"Value: {sample[0][1]}")

[('monica', (-0.4215, 1)), ('chandler', (-0.2048, 1)), ('monica', (0.0, 1)), ('chandler', (0.0, 1)), ('phoebe', (0.0, 1)), ('monica', (0.0, 1)), ('phoebe', (0.2023, 1)), ('ross', (0.0, 1)), ('phoebe', (0.0, 1)), ('monica', (-0.296, 1))]
Key: monica
Value: (-0.4215, 1)


In [80]:
# Implemente e aplique um método reduce para acumulação dos sentimentos dos personagens
def reduce_acc(acc, value) :
  polarity_acc = acc[0]
  count_acc = acc[1]
  polarity_v = value[0]
  count_v = value[1]
  return(polarity_acc + polarity_v, count_acc + count_v)

total = s.reduceByKey(reduce_acc)
print(f"Total Sum Polarity: {total.take(10)}")

Total Sum Polarity: [('gunther', (12.057499999999997, 132)), ('emily', (21.3578, 167)), ('janice', (36.08109999999999, 211)), ('chandler', (1038.7343999999996, 8386)), ('joey', (1071.1549000000005, 8227)), ('richard', (29.971700000000002, 254)), ('monica', (978.2027000000004, 8325)), ('phoebe', (1071.3556999999998, 7365)), ('ross', (1214.3541999999998, 9165)), ('rachel', (1342.4143999999997, 9147))]


In [81]:
# Implemente e aplique um método para calculo do sentimento médio
def average(value):
  sum = value[0]
  count = value[1]
  return(round(sum/count, 5))

avg = total.mapValues(average)
print(avg.take(10))

[('gunther', 0.09134), ('emily', 0.12789), ('janice', 0.171), ('chandler', 0.12387), ('joey', 0.1302), ('richard', 0.118), ('monica', 0.1175), ('phoebe', 0.14547), ('ross', 0.1325), ('rachel', 0.14676)]
